In [ ]:
from pynq import Overlay, allocate
import pynq.lib.dma
import numpy as np

DIM=32
MAX=256

DATA_TYPE = np.uint8

in_buffer_A = allocate(shape=(DIM,DIM), dtype=DATA_TYPE, cacheable=False)
in_buffer_B = allocate(shape=(DIM,DIM), dtype=DATA_TYPE, cacheable=False)
out_buffer_C = allocate(shape=(DIM,DIM), dtype=DATA_TYPE, cacheable=False)

A = np.random.randint(MAX, size=(DIM, DIM), dtype=DATA_TYPE)
B = np.random.randint(MAX, size=(DIM, DIM), dtype=DATA_TYPE)
C = np.zeros((DIM,DIM), dtype=DATA_TYPE)

ol = Overlay('./design_1.bit')
ol.download()

dma0 = ol.axi_dma_0
dma1 = ol.axi_dma_1

np.copyto(in_buffer_A, A)
np.copyto(in_buffer_B, B)
np.copyto(out_buffer_C, C)

def silicon():
    dma0.sendchannel.transfer(in_buffer_A)
    dma1.sendchannel.transfer(in_buffer_B)
    dma0.recvchannel.transfer(out_buffer_C)
    dma0.sendchannel.wait()
    dma1.sendchannel.wait()
    dma0.recvchannel.wait()

    return out_buffer_C

def native():
    return A@B

In [ ]:

print("software execution:")
%time golden = native()

In [ ]:
print("hardware execution:")
%time FPGA_result = silicon()

In [ ]:
err = 1
check = FPGA_result == golden
for row in check:
    for entry in row:
        if entry == False:
            err = 0
if err == 0:
    print("hardware execution failure")
else:
    print("hardware execution successful!")